In [22]:
!pip install -U keras-tuner

In [23]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


In [24]:
!pip install keras-tuner --upgrade

In [25]:
import pandas as pd
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch

In [26]:
data = pd.read_csv("C:/Users/vipul/OneDrive/Desktop/dl dataset/Tetuan City power consumption.csv")

In [27]:
data.head()

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [28]:
data.describe()

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
count,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000
mean,18.810024,68.259518,1.959489,182.696614,75.028022,32344.970564,21042.509082,17835.406218
std,5.815476,15.551177,2.348862,264.400960,124.210949,7130.562564,5201.465892,6622.165099
min,3.247000,11.340000,0.050000,0.004000,0.011000,13895.696200,8560.081466,5935.174070
25%,14.410000,58.310000,0.078000,0.062000,0.122000,26310.668692,16980.766032,13129.326630
50%,18.780000,69.860000,0.086000,5.035500,4.456000,32265.920340,20823.168405,16415.117470
75%,22.890000,81.400000,4.915000,319.600000,101.000000,37309.018185,24713.717520,21624.100420
max,40.010000,94.800000,6.483000,1163.000000,936.000000,52204.395120,37408.860760,47598.326360


In [29]:
X=data.iloc[:,:-1]
Y=data.iloc[:,-1]

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=3,random_state=5)

In [31]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((52413, 7), (52413,), (3, 7), (3,))

In [32]:
Y.shape

(52416,)

In [33]:
x_train= X_train.values.reshape(-1, 1)
x_test = X_test.values.reshape(-1, 1)
y_train= Y_train.values.reshape(-1, 1)
y_test = Y_test.values.reshape(-1, 1)

In [34]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train)

# transform training data
X_train_norm = norm.transform(x_train)

# transform testing dataabs
X_test_norm = norm.transform(x_test)

In [35]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [36]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Power Consumption1')

INFO:tensorflow:Reloading Oracle from existing project project\Power Consumption1\oracle.json
INFO:tensorflow:Reloading Tuner from project\Power Consumption1\tuner0.json


In [37]:
tuner.search_space_summary()

Search space summary
Default search space size: 22
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_6 (Int)
{'default': None, 'co

In [38]:
tuner.search(X_train, Y_train,epochs=5,validation_data=(X_test, Y_test))

INFO:tensorflow:Oracle triggered exit


In [39]:
tuner.results_summary()

Results summary
Results in project\Power Consumption1
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 11
units_0: 512
units_1: 512
learning_rate: 0.0001
units_2: 416
units_3: 448
units_4: 416
units_5: 512
units_6: 256
units_7: 384
units_8: 384
units_9: 416
units_10: 416
units_11: 448
units_12: 416
Score: 1989.2222900390625
Trial summary
Hyperparameters:
num_layers: 20
units_0: 416
units_1: 480
learning_rate: 0.001
units_2: 448
units_3: 128
units_4: 224
units_5: 256
units_6: 32
units_7: 96
units_8: 224
units_9: 160
units_10: 352
units_11: 448
units_12: 64
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
units_18: 32
units_19: 32
Score: 1999.6849772135417
Trial summary
Hyperparameters:
num_layers: 17
units_0: 448
units_1: 448
learning_rate: 0.01
units_2: 288
units_3: 32
units_4: 512
units_5: 288
units_6: 224
units_7: 320
units_8: 192
units_9: 224
units_10: 64
units_11: 352
units_12: 192
units_13: 96
units_14: 352
units_15: 224
units_16: 160
units_17: 320

Classification Problem With GridSearchCV

In [40]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [41]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from sklearn.preprocessing import StandardScaler

In [42]:
# Importing the dataset
dataset = pd.read_csv("C:/Users/vipul/OneDrive/Desktop/dl dataset/Churn_Modelling.csv")
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [43]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [44]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
# onehotencoder = OneHotEncoder(max_categories= [1])
# X = onehotencoder.fit_transform(X).toarray()

from sklearn.compose import ColumnTransformer
columnTransformer = ColumnTransformer([('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(columnTransformer.fit_transform(X),dtype=str)
X = X[:, 1:]

In [45]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [46]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [47]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

C:\Users\vipul\AppData\Local\Temp\ipykernel_4888\3582075148.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [48]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [49]:
grid_result = grid.fit(X_train, y_train)

In [50]:
[grid_result.best_score_,grid_result.best_params_]

[0.8508749961853027,
 {'activation': 'relu',
  'batch_size': 256,
  'epochs': 30,
  'layers': [45, 30, 15]}]

In [51]:
pred_y = grid.predict(X_test)

63/63 [==============================] - 0s 1ms/step


In [52]:
y_pred = (pred_y > 0.5)

In [53]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [54]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [55]:
cm

array([[1523,   72],
       [ 215,  190]], dtype=int64)

In [56]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)

In [57]:
score

0.8565